This model looks at catboosted decision tree, random forest, linear regression

In [23]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [ ]:
# Helper functions

def get_loss(predicted, actual):
    loss = mean_squared_error(prediceted, actual)
    return np.sqrt(loss)

def kfold_validate(X, y, te, model, col):
        scores = np.zeros(k)
    for i, (tr_idx, val_idx) in enumerate(cv.split(X)):
        X_tr, y_tr = X.iloc[tr_idx, :], y[tr_idx]
        X_val, y_val = X.iloc[val_idx, :], y[val_idx]

        model.fit(X_tr, y_tr)
        pred_val = model.predict(X_val)
        scores[i] = get_loss(pred_val, y_val)
        print("========={}-th Fold Score: {}".format(i, scores[i]))
        
        meta_tr[col][val_idx] = pred_val
        meta_tr['val'][val_idx] = y_val

    print("=========Total Score: ", np.mean(scores))
    model.fit(X, y)
    meta_te[col] = model.predict(te)

In [14]:
# Load data
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

# Check data
print("Training set ", train.shape)
print("Test set ", test.shape)

print("NAs in train set\n", train.isnull().sum()[train.isnull().sum() != 0])
print("NAs in test set\n", test.isnull().sum()[test.isnull().sum() != 0])

Training set  (3000, 23)
Test set  (4398, 22)
=======NAs in train set
 belongs_to_collection    2396
genres                      7
homepage                 2054
overview                    8
poster_path                 1
production_companies      156
production_countries       55
runtime                     2
spoken_languages           20
tagline                   597
Keywords                  276
cast                       13
crew                       16
dtype: int64
=======NAs in test set
 belongs_to_collection    3521
genres                     16
homepage                 2978
overview                   14
poster_path                 1
production_companies      258
production_countries      102
release_date                1
runtime                     4
spoken_languages           42
status                      2
tagline                   863
title                       3
Keywords                  393
cast                       13
crew                       22
dtype: int64


In [15]:
# Fill remaining NA's with -1
train.fillna(-1, inplace = True)
test.fillna(-1, inplace = True)

In [21]:
train.iloc[:, :45].describe()
# test.iloc[:, :45].describe()

,id,budget,popularity,runtime,revenue
count,3000.000000,3.000000e+03,3000.000000,3000.000000,3.000000e+03
mean,1500.500000,2.253133e+07,8.463274,107.784000,6.672585e+07
std,866.169729,3.702609e+07,12.104000,22.257189,1.375323e+08
min,1.000000,0.000000e+00,0.000001,-1.000000,1.000000e+00
25%,750.750000,0.000000e+00,4.018053,94.000000,2.379808e+06
50%,1500.500000,8.000000e+06,7.374861,104.000000,1.680707e+07
75%,2250.250000,2.900000e+07,10.890983,118.000000,6.891920e+07
max,3000.000000,3.800000e+08,294.337037,338.000000,1.519558e+09


In [22]:
y = train.revenue
X = train.drop('revenue', axis=1)

In [ ]:
# Cross Validation
k = 10
seed = 10
cross_validation = KFold(n_splits=k, shuffle=True, random_state=seed)

In [ ]:
# CatBoost
cat_boost = CatBoostRegressor(bagging_temperature=0.3, colsample_bylevel=0.7, depth=7, early_stopping_rounds=500, eval_metric='RMSE', iterations=1000, learning_rate=0.01, logging_level='Silent')
cat_boost.fit()